Trend in the funding policies for italian cultural institutes

In Italy every region has different amount of funding and level of participations to cultural things.
This project aims to discover how are the public funding distributed in the last 10 years and how the trend of visitors changes through time.

In [5]:
import pandas as pd

project_20 = pd.read_csv("data/progetti_esteso_CULTURA_TURISMO_2014-2020_20250430.csv", sep=';', low_memory=False)
project_20 = project_20[['CUP','DATA_INIZIO_PREV_STUDIO_FATT','OC_DATA_INIZIO_PROGETTO','OC_DATA_FINE_PROGETTO_EFFETTIVA','DEN_REGIONE','DEN_PROVINCIA','DEN_COMUNE','FINANZ_TOTALE_PUBBLICO','FINANZ_STATO_ESTERO','FINANZ_PRIVATO']]
project_27 = pd.read_csv("data/progetti_esteso_CULTURA_TURISMO_2021-2027_20250430.csv", sep=';', low_memory=False)
project_27 = project_27[['CUP', 'DATA_INIZIO_PREV_STUDIO_FATT','OC_DATA_INIZIO_PROGETTO','OC_DATA_FINE_PROGETTO_EFFETTIVA','DEN_REGIONE','DEN_PROVINCIA','DEN_COMUNE','FINANZ_TOTALE_PUBBLICO','FINANZ_STATO_ESTERO','FINANZ_PRIVATO']]
all_projects = pd.concat([project_20, project_27], ignore_index=True)
all_projects = all_projects.drop_duplicates(subset=['CUP'], keep='first')

In [6]:
date_columns = ['DATA_INIZIO_PREV_STUDIO_FATT','OC_DATA_INIZIO_PROGETTO','OC_DATA_FINE_PROGETTO_EFFETTIVA']

all_projects[date_columns] = (all_projects[date_columns].apply(lambda col: pd.to_datetime(col.astype('Int64').astype(str), format='%Y%m%d', errors='coerce')))

all_projects = all_projects.sort_values(by=date_columns, ascending=[True, True, True], na_position='last').reset_index(drop=True)
all_projects.to_csv("data/open_coesione.csv", index=False, encoding="utf-8")

In [7]:
import pandas as pd
import os
import glob

def transform_table(file):
    year = os.path.basename(file).split("_")[2]
    #detect header of table
    with open(file, "r", encoding = "latin-1") as f:
        for i,line in enumerate(f):
            if line.startswith("Regione"):
                header_row = i
                break
    visitors = pd.read_csv(file, skiprows = header_row, sep=";", encoding="latin-1")
    visitors.rename(columns={"Totale complessivo":"Totale_regione"}, inplace= True) 
    # set 'Regione' as columns, months as rows
    visitors = visitors.set_index("Regione").T
    
    visitors.dropna(axis=1, how="all", inplace= True)
    visitors.dropna(axis=0, how="all", inplace= True)
    
    # rename the rows to add the year
    visitors.index = [f"{month}_{year}" for month in visitors.index]

    return visitors

In [8]:
# apply to all visitatori files
files = glob.glob("data/musei_tavola5_*_visitatori.csv")
total_visitors = []
for f in files:
    total_visitors.append(transform_table(f))
    
# merge along rows
output_visitors = pd.concat(total_visitors)
output_visitors.to_csv("data/mic_visitors.csv", encoding="utf-8")

# apply to all introiti files
tables_income = glob.glob("data/musei_tavola5_*_introiti.csv")
total_income = []
for i in tables_income:
    total_income.append(transform_table(i))
    
# merge along rows
output_income = pd.concat(total_income)
output_income.to_csv("data/mic_income.csv", encoding="utf-8")
